<a href="https://colab.research.google.com/github/MariMurotani/ColabNotebooks/blob/main/CEVR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/CEVR

/content/drive/MyDrive/Colab Notebooks/CEVR


In [ ]:
!git clone https://github.com/skchen1993/2023_CEVR.git
%cd 2023_CEVR

fatal: destination path '2023_CEVR' already exists and is not an empty directory.
/content/drive/MyDrive/Colab Notebooks/CEVR/2023_CEVR


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/CEVR/2023_CEVR

/content/drive/MyDrive/Colab Notebooks/CEVR/2023_CEVR


In [ ]:
!pip install torch torchvision
!pip install numpy opencv-python pillow tqdm matplotlib
!pip install pytorch-msssim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 139.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 104.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvji

In [ ]:
# Bモデル側
!cp /content/drive/MyDrive/Colab\ Notebooks/CEVR/2023_CEVR/train_strategy/experiment/CEVR_NormNoAffine_Maps_GN_Bmodel/inc/final_model.pth \
     /content/drive/MyDrive/Colab\ Notebooks/CEVR/2023_CEVR/train_strategy/experiment/CEVR_NormNoAffine_Maps_GN_Bmodel/inc/model_final.pth

# Dモデル側
!cp /content/drive/MyDrive/Colab\ Notebooks/CEVR/2023_CEVR/train_strategy/experiment/CEVR_NormNoAffine_Maps_GN_Dmodel/dec/final_model.pth \
     /content/drive/MyDrive/Colab\ Notebooks/CEVR/2023_CEVR/train_strategy/experiment/CEVR_NormNoAffine_Maps_GN_Dmodel/dec/model_final.pth

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import os
from core.HDR_model import build_network
from core.utils import save_fig
import torch.nn.functional as F

def pad_to_multiple_of_64(tensor):
    b, c, h, w = tensor.size()
    pad_h = (64 - h % 64) % 64
    pad_w = (64 - w % 64) % 64
    return F.pad(tensor, (0, pad_w, 0, pad_h), mode='reflect'), (pad_h, pad_w)

def crop_back(tensor, pad_h, pad_w):
    if pad_h == 0 and pad_w == 0:
        return tensor
    return tensor[:, :, :-pad_h or None, :-pad_w or None]

# ===== ユーザー設定 =====
input_path = "img/IMG_5202_780.jpg"
output_dir = "./ev_outputs_original"
os.makedirs(output_dir, exist_ok=True)
ev_list = [-2.0, -1.0, 1.0, 2.0]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ===== 前処理：パディングありで元画像サイズ保持 =====
img = Image.open(input_path).convert("RGB")
orig_size = img.size
input_tensor = transforms.ToTensor()(img).unsqueeze(0).to(device)
input_tensor, (pad_h, pad_w) = pad_to_multiple_of_64(input_tensor)

# ===== モデル構築（EV_info = 1前提）=====
class Args:
    model_name = "CEVR_NormNoAffine_Maps"
    decode_name = "mult_resizeUp_map"
    act = "leaky_relu"
    mlp_num = 3
    pretrain = "vgg"
    cycle = False
    sep = 0.5
    EV_info = 1
    init_weight = False
    norm_type = "GroupNorm"
    NormAffine = False

args = Args()
model_inc = build_network(args).to(device)
model_dec = build_network(args).to(device)

# ===== 重み読み込み =====
model_inc.load_state_dict(torch.load("train_strategy/experiment/CEVR_NormNoAffine_Maps_GN_Bmodel/inc/model_final.pth"))
model_dec.load_state_dict(torch.load("train_strategy/experiment/CEVR_NormNoAffine_Maps_GN_Dmodel/dec/model_final.pth"))
model_inc.eval()
model_dec.eval()

# ===== 推論ループ =====
for ev in ev_list:
    step = torch.tensor([[ev]], dtype=torch.float32).to(device)
    ori = torch.tensor([[0.0]], dtype=torch.float32).to(device)

    with torch.no_grad():
        if ev > 0:
            output = model_inc(input_tensor, step, ori)
        else:
            output = model_dec(input_tensor, step, ori)

    # パディング分をカットして元サイズに戻す
    output = crop_back(output, pad_h, pad_w)

    # 保存
    output_img = output.squeeze(0).cpu()
    out_path = os.path.join(output_dir, f"EV{int(ev):+d}.png")
    save_fig(output_img, out_path)
    print(f"✅ 出力: {out_path}（元画像サイズ: {orig_size}）")

!!!! net_name = CEVR_NormNoAffine_Maps !!!!
CEVR(Maps) Normalization layer affine: False
BottleNeck normlization don't use affine(Learnable parameter)
#############################self.norm_type in bottleneck:  GroupNorm  ####################
Decoder normlization don't use affine(Learnable parameter)
#############################self.norm_type in DecoderBlock(Maps):  GroupNorm  ####################
Resize_conv upsample mode:  bicubic
Decoder normlization don't use affine(Learnable parameter)
#############################self.norm_type in DecoderBlock(Maps):  GroupNorm  ####################
Resize_conv upsample mode:  bicubic
Decoder normlization don't use affine(Learnable parameter)
#############################self.norm_type in DecoderBlock(Maps):  GroupNorm  ####################
Resize_conv upsample mode:  bicubic
model_name: CEVR_NormNoAffine_Maps pretrain: vgg mlp_num: 3 decoder: mult_resizeUp_map activation: leaky_relu
!!!! net_name = CEVR_NormNoAffine_Maps !!!!
CEVR(Maps) Normali

In [ ]:
import torch
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import os
from core.HDR_model import build_network
from core.utils import save_fig

# ユーザー設定
input_path = "img/IMG_5202.jpg"
output_dir = "./IMG_5202_blended"
os.makedirs(output_dir, exist_ok=True)
ev_list = [-2.0, -1.0, 1.0, 2.0]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
patch_height, patch_width = 384, 512
overlap = 64
stride_h, stride_w = patch_height - overlap, patch_width - overlap

# モデル構築
class Args:
    model_name = "CEVR_NormNoAffine_Maps"
    decode_name = "mult_resizeUp_map"
    act = "leaky_relu"
    mlp_num = 3
    pretrain = "vgg"
    cycle = False
    sep = 0.5
    EV_info = 1
    init_weight = False
    norm_type = "GroupNorm"
    NormAffine = False

args = Args()
model_inc = build_network(args).to(device)
model_dec = build_network(args).to(device)
model_inc.load_state_dict(torch.load("train_strategy/experiment/CEVR_NormNoAffine_Maps_GN_Bmodel/inc/model_final.pth"))
model_dec.load_state_dict(torch.load("train_strategy/experiment/CEVR_NormNoAffine_Maps_GN_Dmodel/dec/model_final.pth"))
model_inc.eval()
model_dec.eval()

# 重みマスク（中心重視の窓関数）
def create_weight_mask(h, w):
    y = torch.linspace(0, 1, h).view(-1, 1)
    x = torch.linspace(0, 1, w).view(1, -1)
    w = torch.minimum(y, 1 - y) * torch.minimum(x, 1 - x)
    w = 0.1 + 0.9 * (w / w.max())  # ←ここ
    return w

# 画像読み込み
img = Image.open(input_path).convert("RGB")
input_tensor = transforms.ToTensor()(img).unsqueeze(0).to(device)
_, _, full_h, full_w = input_tensor.shape
weight_mask = create_weight_mask(patch_height, patch_width).to(device).unsqueeze(0).repeat(3,1,1)

# 推論ループ
for ev in ev_list:
    step = torch.tensor([[ev]], dtype=torch.float32).to(device)
    ori = torch.tensor([[0.0]], dtype=torch.float32).to(device)

    output_accum = torch.zeros_like(input_tensor)
    weight_sum = torch.zeros_like(input_tensor)

    with torch.no_grad():
        for i in range(0, full_h, stride_h):
            for j in range(0, full_w, stride_w):
                patch = input_tensor[:, :, i:i+patch_height, j:j+patch_width]
                orig_h, orig_w = patch.size(2), patch.size(3)

                # パディング
                pad_h = patch_height - orig_h
                pad_w = patch_width - orig_w
                if pad_h > 0 or pad_w > 0:
                    patch = F.pad(patch, (0, pad_w, 0, pad_h), mode="replicate")

                out = model_inc(patch, step, ori) if ev > 0 else model_dec(patch, step, ori)
                out = out[:, :, :orig_h, :orig_w]
                wm = weight_mask[:, :orig_h, :orig_w].unsqueeze(0)

                output_accum[:, :, i:i+orig_h, j:j+orig_w] += out * wm
                weight_sum[:, :, i:i+orig_h, j:j+orig_w] += wm

    output_tensor = output_accum / (weight_sum + 1e-8)
    output_img = output_tensor.squeeze(0).cpu()
    save_fig(output_img, os.path.join(output_dir, f"EV{int(ev):+d}.png"))
    print(f"✅ EV{ev:+.1f} done.")

!!!! net_name = CEVR_NormNoAffine_Maps !!!!
CEVR(Maps) Normalization layer affine: False
BottleNeck normlization don't use affine(Learnable parameter)
#############################self.norm_type in bottleneck:  GroupNorm  ####################
Decoder normlization don't use affine(Learnable parameter)
#############################self.norm_type in DecoderBlock(Maps):  GroupNorm  ####################
Resize_conv upsample mode:  bicubic
Decoder normlization don't use affine(Learnable parameter)
#############################self.norm_type in DecoderBlock(Maps):  GroupNorm  ####################
Resize_conv upsample mode:  bicubic
Decoder normlization don't use affine(Learnable parameter)
#############################self.norm_type in DecoderBlock(Maps):  GroupNorm  ####################
Resize_conv upsample mode:  bicubic
model_name: CEVR_NormNoAffine_Maps pretrain: vgg mlp_num: 3 decoder: mult_resizeUp_map activation: leaky_relu
!!!! net_name = CEVR_NormNoAffine_Maps !!!!
CEVR(Maps) Normali